In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')
train = train[train["Cover_Type"]!=5]  # Label 5 occurs only 1 time and creates problem in some areas

In [ ]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier

In [ ]:
def objective(trial):   
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    param = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1),
        "depth": trial.suggest_int("depth", 4, 10), 
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 10), 
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 0.01, 1),
    }
    clf = CatBoostClassifier(**param, verbose=0, task_type='GPU', objective= "MultiClass")
    clf.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=100)
    return accuracy_score(y_test, clf.predict(X_test))

y = train["Cover_Type"]
X = train.drop(['Id',"Cover_Type"], axis=1)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=500, timeout=3000)


In [ ]:
y_train = train["Cover_Type"]
X_train = train.drop(['Id',"Cover_Type"], axis=1)
X_test = test.drop(['Id'], axis=1)

clf = CatBoostClassifier(**study.best_params, verbose=0, task_type="GPU", objective= "MultiClass")
clf.fit(X_train, y_train)
sample_submission.iloc[:,1] = clf.predict(X_test)
sample_submission.to_csv('submission_CatBoostClassifier.csv',index=False)
sample_submission.head()